In [6]:
import numpy as np
import warnings
import pandas as pd
import typing as tp
from sklearn import preprocessing
import xgboost as xgb

warnings.filterwarnings("ignore", category=UserWarning)
xgb.set_config(verbosity = 0)

# Feature Engineering

In [18]:
# Label encoder. This func helps to preprocess categorical variables.
def preprocessing_cat_features_le(cat_features: list, df: pd.DataFrame) -> tp.Tuple[pd.DataFrame, dict]:
    df = df.copy()

    maps = {}
    for feature in cat_features:
        le = preprocessing.LabelEncoder()
        df[feature] = le.fit_transform(df[feature].astype(str))
        maps[feature] = le
    return df, maps

# Function for preprocessing a training dataset with a target variable and a test dataset without a target variable
def preprocessing_data(train: pd.DataFrame, test: pd.DataFrame, target: str, var_for_preprocessing: dict) \
        -> tp.Tuple[pd.DataFrame, pd.DataFrame, dict, list]:

    # Data imputation
    train = train.fillna(-1)
    test = test.fillna(-1)

    # Join training and test dataset
    df_full = pd.concat([train, test])

    # Add some feature to black list, witch will be removed from main dataset
    black_list = var_for_preprocessing.get("feature_black_list")
    if black_list:
        black_list.append(target)
    else:
        black_list = [target]

    for var_type, var in var_for_preprocessing.items():
        # Preprocessing categorical and numeric categorical features
        if var_type == "categorical_features" or var_type == "numeric_categorical_features":
            for feature, n in var:
                set_top_n_categories_in_variable(df_full, feature, n)
        # Normalisation
        elif var_type == "continuous_numeric_features":
            df_full[var]= df_full[var].apply(lambda x: (x - x.mean())/x.std())
        # One Hot Encoding
        elif var_type == "one_hot_encoding":
            df_bin_var = pd.get_dummies(df_full[var], drop_first=True)
            black_list.extend(var)
            df_full = pd.concat([df_full, df_bin_var], axis=1)
    # Select categorical features
    cat_features = list(set(df_full.select_dtypes(exclude='number').columns.tolist())-set(black_list))
    # Label encoding
    new_df_full, maps = preprocessing_cat_features_le(cat_features, df_full)
    # The features, that will be used for modeling.
    features = list(set(new_df_full.columns.tolist())-set(black_list))

    df_test = new_df_full.loc[new_df_full.notified.isna()][features]
    df_train = new_df_full.loc[new_df_full.notified.notna()][features+[target]]
    return df_train, df_test, maps, features

# Function to set top N categories in given categorical variable
def set_top_n_categories_in_variable(df: pd.DataFrame, feature: str, n_cat: int):
    top_n_var = df[feature].value_counts().nlargest(n_cat).index
    df[feature] = df[feature].where(df[feature].isin(top_n_var), other='Other')

In [19]:
df_train = pd.read_csv('input/cybersecurity_training.csv', sep='|', index_col=0)
df_test = pd.read_csv('input/cybersecurity_test.csv', sep='|', index_col=0)
print(df_train.shape)
print(df_test.shape)
df_full = pd.concat([df_train, df_test])

(39427, 62)
(20000, 61)


In [20]:
# Selected categor
categorical_features = [
    ("categoryname", 7),
    ("ipcategory_name", 5),
    ("ipcategory_scope", 2),
    ("dstipcategory_dominate", 4),
    ("srcipcategory_dominate", 4)
]

numeric_categorical_features = [
    ("parent_category", 2),
    ("overallseverity", 3),
    ("alerttype_cd", 3),
    ("direction_cd", 2),
    ("eventname_cd", 4),
    ("severity_cd", 3),
    ("devicetype_cd", 2),
    ("devicevendor_cd", 2),
    ("srcipcategory_cd", 3),
    ("dstipcategory_cd", 2),
    ("trustscore", 2),
    ("dstportcategory_dominate", 3),
    ("username_cd", 8),
    ("protocol_cd", 4),
    ("reportingdevice_cd", 5),
    ("dstport_cd", 5),
    ("srcport_cd", 10),
    ("dstip_cd", 6),
    ("srcip_cd", 10),
    ("p6", 3),
    ("p9", 1),
    ("p5m", 3),
    ("p5w", 2),
    ("p5d", 2),
    ("p8w", 2),
    ("p8m", 3),
    ("p8d", 2)
]

numeric_continue_features = ["thrcnt_month", "thrcnt_week", "thrcnt_day", "timestamp_dist", "correlatedcount"]
feature_black_list = ['grandparent_category']
smt_to_do = "ip"
one_hot_encoding = ["weekday"]
features_for_preprocessing = {"categorical_features": categorical_features,
                              "numeric_categorical_features": numeric_categorical_features,
                              # "numeric_continue_features": numeric_continue_features,
                              "feature_black_list": feature_black_list,
                              "one_hot_encoding": one_hot_encoding}

# Select target variable
target = 'notified'

In [21]:
df_train, df_test, *_ =  preprocessing_data(df_train, df_test, target, features_for_preprocessing)
df_train.head()

,start_hour,weekday_Sat,overallseverity,weekday_Thu,thrcnt_day,untrustscore,severity_cd,n6,timestamp_dist,p5d,...,n1,alerttype_cd,devicevendor_cd,p8w,devicetype_cd,p5w,dstport_cd,weekday_Mon,flowscore,notified
alert_ids,,,,,,,,,,,,,,,,,,,,,
Nhq,8,0,0,0,675,2,1,-1.0,65684,0,...,-1.0,1,1,0,1,0,2,0,3,0.0
XZt,1,0,2,1,2,5,3,-1.0,1188030,1,...,-1.0,3,1,0,1,1,5,0,5,0.0
bBz,14,0,1,0,628,4,1,-1.0,43716,0,...,-1.0,1,1,1,1,1,2,0,3,0.0
ZNr,20,0,1,0,96,5,0,-1.0,0,1,...,-1.0,0,0,1,0,1,0,0,3,0.0
poV,14,1,1,0,632,5,1,0.0,2401,0,...,0.0,1,1,0,1,0,1,0,3,0.0


In [22]:
df_test.head()

,start_hour,weekday_Sat,overallseverity,weekday_Thu,thrcnt_day,untrustscore,severity_cd,n6,timestamp_dist,p5d,...,correlatedcount,n1,alerttype_cd,devicevendor_cd,p8w,devicetype_cd,p5w,dstport_cd,weekday_Mon,flowscore
alert_ids,,,,,,,,,,,,,,,,,,,,,
Slg,11,0,0,0,42,3,1,1.0,0,0,...,1,0.0,1,1,0,1,0,1,0,3
WKM,22,1,2,0,3,5,1,0.0,0,0,...,1,0.0,1,1,0,1,0,1,0,3
dkm,7,1,0,0,602,2,1,0.0,0,0,...,1,0.0,1,1,0,1,0,1,0,3
RIX,0,0,0,1,4,2,1,1.0,0,0,...,1,0.0,1,1,0,1,0,1,0,3
qFU,18,0,0,0,20,3,1,1.0,258273,0,...,14,0.0,1,1,0,1,0,1,0,3
